In [98]:
# read the hotel reservation file
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import numpy as np

hotel = pd.read_csv('Hotel Reservations.csv', header=None)
del hotel[hotel.columns[0]]
hotel = hotel[1:]

C:\Users\cezar\AppData\Local\Temp\ipykernel_12020\3197420233.py:9: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  hotel = pd.read_csv('Hotel Reservations.csv', header=None)


In [ ]:
hotel.info()

In [ ]:
categorical_columns = [
    "type_of_meal_plan",
    "required_car_parking_space",
    "room_type_reserved",
    "market_segment_type",
    "repeated_guest",
    "booking_status"
]

hotel = hotel.drop(['Booking_ID'], axis=1)  

numerical_columns = hotel.columns.difference(categorical_columns)


In [99]:
#implement association rules for every column
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

def discretize(df, column, bins, labels):
    df[column] = pd.cut(df[column], bins=bins, labels=labels)

def association_rules_for_column(hotel, column):
    if column == 'booking_status':
        return
    
    df = hotel.groupby([column, 'booking_status']).size().reset_index(name='counts')
    df = df.pivot(index=column, columns='booking_status', values='counts').fillna(0)
    df['total'] = df['Canceled'] + df['Not_Canceled']
    df['canceled_ratio'] = df['Canceled'] / df['total']
    df['not_canceled_ratio'] = df['Not_Canceled'] / df['total']
    df = df.sort_values(by='canceled_ratio', ascending=False)
    return df

#discretize average price
bins = [i * 25 for i in range(24)]
labels =  ["P" + str(i * 25) + "-" + str((i+1) * 25) for i in range(23)]

hotel[16] = hotel[16].astype('float64')
discretize(hotel, 16, bins=bins, labels=labels)
hotel[16] = hotel[16].astype('object')

bins = [i * 25 for i in range(18)]
labels =  ["L" + str(i * 25) + "-" + str((i+1) * 25) for i in range(17)]
hotel[8] = hotel[8].astype('int64')
discretize(hotel, 8, bins=bins, labels=labels)
hotel[8] = hotel[8].astype('object')

col_abvr = ['AD_', 'CH_', 'WEN_', "WKN_", "", "PARK_", "", "", "", "MON_", "DATE_", "", "REP_", "PC_", "PNC_", "", "SPEC_", ""]
print(len(col_abvr))
print(hotel.shape[1])
for i in range(len(col_abvr)):
    hotel.iloc[:,i] = col_abvr[i] + hotel.iloc[:,i].astype(str)
    



18
18


## Association rules for all attributes (support/confidence experiments)

In [114]:
from apyori import apriori

print(hotel.shape[0])
print(hotel.shape[1])

# Intializing the list
transacts = []
rows = hotel.shape[0]
columns = hotel.shape[1]
for i in range(rows):
    transacts.append([str(hotel.values[i,j]) for j in range(0, columns)])

print("TRANSCATIONS : ", len(transacts) * len(transacts[0]))
    
rule = apriori(transactions = transacts, min_support = 0.003, min_confidence = 0.1, min_lift = 2, min_length = 2, max_length = 2)
output = list(rule) # returns a non-tabular output

def inspect(output):
    lhs         = [tuple(result[2][0][0])[0] for result in output]
    rhs         = [tuple(result[2][0][1])[0] for result in output]
    support    = [result[1] for result in output]
    confidence = [result[2][0][2] for result in output]
    lift       = [result[2][0][3] for result in output]
    return list(zip(lhs, rhs, support, confidence, lift))

output_DataFrame = pd.DataFrame(inspect(output), columns = ['Left_Hand_Side', 'Right_Hand_Side', 'Support', 'Confidence', 'Lift'])
output_DataFrame

36275
18
TRANSCATIONS :  652950


,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,2017,MON_10,0.052736,0.293675,2.003586
1,2017,Meal Plan 2,0.035121,0.195579,2.146632
2,AD_0,CH_2,0.003666,0.956835,32.806401
3,AD_0,Room_Type 2,0.003666,0.956835,50.157764
4,Aviation,AD_1,0.003391,0.984000,4.638674
...,...,...,...,...,...
111,REP_1,nan,0.006864,0.267742,5.565810
112,WEN_1,WKN_0,0.041957,0.152276,2.314125
113,WEN_2,WKN_5,0.026823,0.107265,2.410802
114,WKN_6,WEN_2,0.003473,0.666667,2.666005


In [116]:
output_DataFrame.nlargest(n = 10, columns = 'Support')

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
95,Meal Plan 2,Offline,0.065196,0.715582,2.465592
98,P125-150,Room_Type 4,0.053563,0.394358,2.361784
0,2017,MON_10,0.052736,0.293675,2.003586
15,AD_3,Room_Type 4,0.047884,0.749676,4.489765
96,Meal Plan 2,P100-125,0.046919,0.514977,2.021076
6,AD_1,Corporate,0.043170,0.203509,3.660030
112,WEN_1,WKN_0,0.041957,0.152276,2.314125
36,Corporate,L0-25,0.035975,0.647000,2.367592
1,2017,Meal Plan 2,0.035121,0.195579,2.146632
25,L175-200,Canceled,0.029139,0.707023,2.157953
